In [1]:
import os
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset as BaseDataset
from torch.utils.data import DataLoader as BaseDataLoader

from transformers import GPT2Tokenizer, AutoModelForCausalLM, GPT2LMHeadModel, AutoTokenizer
from transformers import TrainingArguments, Trainer

from datasets import load_dataset

device="cuda"

/home/mnasiri/masoud_nasiripour/second_phase/gpt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("bolbolzaban/gpt2-persian")

In [3]:
config = {
    "emb_dim" : 1024,
    "letter_emb_dim": 1024,
    "vocab_size" : tokenizer.vocab_size,
    "save_path": "./models/v2.pth"
}

In [4]:
class Dataset(BaseDataset):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.data = load_dataset("csv", data_files="dataset.csv")["train"]

    def __getitem__(self, ix):
        item = self.data[ix]
        return item


    def __len__(self, ):
        return len(self.data)


In [5]:
class CustomCollator:
    def __call__(self, batch):
        titles = [item["title"] for item in batch if item is not None]
        ctx_embs = torch.tensor([json.loads(item["context_embedding"]) for item in batch if item is not None], dtype=torch.float)

        tokenized_title = tokenizer(titles,
                  padding="max_length",
                  truncation=True,
                  return_tensors="pt",
                  max_length=256)
        
        attention_mask = torch.stack([torch.cat([torch.tensor([1,], dtype=torch.long), mask], dim=-1) for mask in tokenized_title["attention_mask"]]).to(device)

        input_ids = tokenized_title["input_ids"][:, :-1].long()
        targets = tokenized_title["input_ids"]
        targets = targets.masked_fill(targets == tokenizer.pad_token_id, -100)

        return {
            "attention_mask": attention_mask[:, :-1].to(device),
            "letter_emb": ctx_embs.to(device),
            "input_ids": input_ids.to(device),
            "label": targets.to(device)
        }

dataset = Dataset(tokenizer)
collator_fn = CustomCollator()

In [6]:
class Model(nn.Module):

    def __init__(self, tokenizer, config):
        super().__init__()
        self.tokenizer = tokenizer
        self.letter_projection = nn.Sequential(nn.Linear(config["letter_emb_dim"], config["letter_emb_dim"] // 2),
                                                nn.Linear(config["letter_emb_dim"] // 2, config["emb_dim"]))
        self.gpt = GPT2LMHeadModel.from_pretrained("bolbolzaban/gpt2-persian")

        for p in self.gpt.parameters():
            p.requires_grad=False
        for p in self.gpt.lm_head.parameters():
            p.requires_grad=True


    @classmethod
    def from_pretrained(cls, tokenizer, config):
        print("check model existance...")
        if os.path.isfile(config["save_path"]):
            print("Loading the model...")
            self = cls(tokenizer, config)
            self.load_state_dict(torch.load(config["save_path"], weights_only=True))
            print("loaded successfully!")
        else:
            print(f"couldn't find the {config['save_path']} file!")
            print("Creating a new model...")
            self = cls(tokenizer, config)
        return self

    def save(self, ):
        torch.save(self.state_dict(), config["save_path"])
        print(f"Model saved at {config['save_path']}!")
    
    def forward(self, attention_mask, letter_emb, input_ids, label):
        letter_emb = self.letter_projection(letter_emb).unsqueeze(1)
        x = self.gpt.transformer.wte(input_ids)
        x += self.gpt.transformer.wpe(torch.arange(x.shape[1]).to(device))
        x = torch.cat([letter_emb, x], dim=1)
        output = self.gpt(inputs_embeds=x,
            attention_mask=attention_mask,
            return_dict=True,
            labels=label
        )
        return output

    
    @torch.no_grad
    def generate(self, letter_emb):
        model.eval()
        letter_emb = torch.tensor(json.loads(letter_emb)).view(1,1,-1).to(device)
        letter_emb = self.letter_projection(letter_emb)
        output = model.gpt.generate(
        inputs_embeds=letter_emb,
        attention_mask=torch.ones((1, 1), dtype=torch.long).to(device),
        do_sample=True,
        top_p=0.9,
        temperature=0.9,
        num_beams=5,
        max_length=128,
        min_length=1,
        repetition_penalty=1.0,
        length_penalty=1.0,
        num_return_sequences=1,)
        return self.tokenizer.batch_decode(output)

model = Model.from_pretrained(tokenizer, config)
model.to(device);

check model existance...
couldn't find the ./models/v2.pth file!
Creating a new model...


In [7]:
train_args = TrainingArguments(
    output_dir="./cache/",
    learning_rate=1e-3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=16,
    num_train_epochs=30,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    save_strategy="epoch",
    logging_steps=10,
    logging_strategy="steps",
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    save_safetensors=False,
)

trainer = Trainer(model=model,
        args=train_args,
        data_collator=collator_fn,
        train_dataset=dataset)

In [8]:
trainer.train()
model.save()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,7.637200
20,6.024400
30,5.241800
40,4.557900
50,4.433100
60,4.259400
70,4.041000
80,3.559100
90,3.685100
100,3.609300


Model saved at ./models/v2.pth!


In [ ]:
''.join(model.generate(dataset[0]["context_embedding"]), )

In [9]:
ix = 10
sample_data = dataset[ix]
with torch.no_grad():
    model.eval()
    letter_emb = model.letter_projection(torch.tensor(json.loads(sample_data["context_embedding"])).to(device).view(1, 1,-1))
    output = model.gpt.generate(
        inputs_embeds=letter_emb,
        attention_mask=torch.ones((1, 1), dtype=torch.long).to(device),
        do_sample=True,
        top_p=0.9,
        temperature=0.9,
        num_beams=5,
        max_length=256,
        min_length=1,
        repetition_penalty=1.0,
        length_penalty=1.0,
        num_return_sequences=1,
    )

    output_ids = tokenizer.batch_decode(output, skip_special_tokens=True)
    print('Generated title:', ''.join(output_ids))
    print('True title:', sample_data["title"])
    print('context:', sample_data["context"])

Setting `pad_token_id` to `eos_token_id`:9 for open-end generation.


Generated title: باز باز باز گذر باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز باز بازیابی باز باز باز باز باز باز باز باز باز‌ باز باز باز پیش باز‌ پیش باز باز باز‌ باز‌ پیش‌ پیش‌‌‌‌ باز‌‌‌‌ گذر پیش‌‌ پیش‌‌‌‌ گذر‌‌‌‌‌‌‌‌‌‌‌‌ گذر‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌ باز‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌ پاسخ‌‌‌ باز‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌ پاسخ‌‌‌‌‌‌‌‌‌‌‌
True title: بازنشانی گذرواژه سامانه اتوماسیون اداری
context: 

شماره: 

تاریخ: 

پیوست: 
دارد
بسمه تعالی




جناب آقای مهندس محمد شیخ
مدیر عامل محترم شرکت تحلیلگران هوشمند فناوری اطلاعات امید

با سلام و احترام
بازگشت به نامه شماره ۵۸۵۵ ‏‏‏- ۰۳ ‏‏‏- ۱ مورخ ۱۵ ‏‏‏ / ۰۷ ‏‏‏ / ۱۴۰۳ در خصوص بازنشانی گذرواژه‏های سامانه اتوماسیون اداری به استحضار می‏رساند حسب هماهنگی‏های بعمل آمده عملیات مذکور در روز چهارشنبه مورخ ۰۲ ‏‏‏‏‏‏‏‏‏‏‏‏‏‏ / ۰۸ ‏‏‏‏‏‏‏‏‏‏‏‏‏‏ / ۱۴۰۳ توسط